In [17]:
import numpy as np
import os, random
import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler, QuantileTransformer
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from az_model import *

random.seed(1)

In [18]:
save_path = '/home/mr6564/continual_research/AZ_Data/Family_Transformed/'

tr_file_save = save_path + 'Family_AZ_Train_Transformed.npz'
te_file_save = save_path + 'Family_AZ_Test_Transformed.npz'

tr_data = np.load(tr_file_save, allow_pickle=True)
te_data = np.load(te_file_save, allow_pickle=True)
X_train, Y_train, Y_tr_family = tr_data['X_train'], tr_data['Y_train'], tr_data['Y_tr_family']
X_test, Y_test, Y_te_family = te_data['X_test'], te_data['Y_test'], te_data['Y_te_family']

In [19]:
X_train.shape, X_test.shape

((257023, 2439), (28559, 2439))

In [54]:
class AZ_MLP_Net(nn.Module):
    def __init__(self, input_features, n_classes):
        super(AZ_MLP_Net, self).__init__()
        self.input_feats_length = input_features
        self.output_classes = n_classes
        
        
        self.fc01 = nn.Linear(self.input_feats_length, 2048)
        self.fc01_bn = nn.BatchNorm1d(2048)
        self.fc01_drop = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(2048, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.fc1_drop = nn.Dropout(p=0.5)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc2_bn = nn.BatchNorm1d(512)
        self.fc2_drop = nn.Dropout(p=0.5)
        
        self.fc3 = nn.Linear(512, 256)
        self.fc3_bn = nn.BatchNorm1d(256)
        self.fc3_drop = nn.Dropout(p=0.5)        
        
        self.fc4 = nn.Linear(256, 128)
        self.fc4_bn = nn.BatchNorm1d(128)
        self.fc4_drop = nn.Dropout(p=0.5)  
        
        self.fc_last = nn.Linear(128, self.output_classes) 
        
        self.activate = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        #print(x.shape)
        
        x = self.fc01(x)
        x = self.fc01_bn(x)
        x = self.activate(x) 
        x = self.fc01_drop(x)
        
        
        x = self.fc1(x)
        x = self.fc1_bn(x)
        x = self.activate(x) 
        x = self.fc1_drop(x)

        x = self.fc2(x)
        x = self.fc2_bn(x)
        x = self.activate(x) 
        x = self.fc2_drop(x)
        
        x = self.fc3(x)
        x = self.fc3_bn(x)
        x = self.activate(x) 
        x = self.fc3_drop(x)
        
        x = self.fc4(x)
        x = self.fc4_bn(x)
        x = self.activate(x) 
        x = self.fc4_drop(x)
        
        x = self.fc_last(x)
        return x

num_exps = 1 #args.num_exps
num_epoch = 500 #args.num_epoch
batch_size = 2048 #args.batch_size
#replay_portion = 1.0 #args.replay_portion
patience = 5
replay_type = 'static_training'

exp_seeds = [random.randint(1, 99999) for i in range(num_exps)]



cnt =  1    
for exp in exp_seeds:
    start_time = time.time()
    
    current_task = exp
    
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = AZ_MLP_Net(X_train.shape[1], 100)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.CrossEntropyLoss() #weight=class_weights.to(device)
    
    
    standardization = StandardScaler()
    standard_scaler = standardization.partial_fit(X_train)
    
    # X_train, y_train, X_valid, Y_valid, X_test, Y_test
    
    X_train = standard_scaler.transform(X_train)
    X_test = standard_scaler.transform(X_test)


    X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
    X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)   
    
    model_save_dir = '../../drebin_exps_store/MLP_model/statictraining_' + str(current_task) + '/'
    create_parent_folder(model_save_dir)

    results_save_dir = '../../drebin_exps_store/MLP_Model/statictraining_results/' 
    create_parent_folder(results_save_dir)

        
    task_training_time, epoch_ran, training_loss, validation_loss  =\
                            training_early_stopping(model, model_save_dir,\
                             X_train, Y_train,\
                            X_test, Y_test, patience, batch_size, device, optimizer, num_epoch,\
                             criterion, replay_type, current_task, exp, earlystopping=True)

    acc = testing_aucscore(model, X_test, Y_test, batch_size, device)   

    

    end_time = time.time()
    cnt += 1
    print(f'Elapsed time {(end_time - start_time)/60} mins.')
    

Torch 2.0.1 CUDA 11.8
Model has 7.805156m parameters
Epoch 1 of 500


100%|███████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 16.85it/s]


Train Loss: 5997.9397, Train Acc: 39.33
Val Loss: 2310.3396, Val Acc: 74.64
Validation loss decreased (inf --> 2310.339589).  Saving model ...
../../drebin_exps_store/MLP_model/statictraining_15456/best_model_epoch_1.pt
Epoch 2 of 500


100%|███████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 14.27it/s]


Train Loss: 2507.0093, Train Acc: 72.11
Val Loss: 1401.7952, Val Acc: 82.43
Validation loss decreased (2310.339589 --> 1401.795239).  Saving model ...
../../drebin_exps_store/MLP_model/statictraining_15456/best_model_epoch_2.pt
Epoch 3 of 500


...

Train Loss: 730.1046, Train Acc: 90.30
Val Loss: 705.5743, Val Acc: 90.93
EarlyStopping counter: 4 out of 5
Epoch 40 of 500


100%|███████████████████████████████████████████████████████████████| 14/14 [00:01<00:00, 13.59it/s]


Train Loss: 726.3851, Train Acc: 90.16
Val Loss: 668.3496, Val Acc: 91.36
EarlyStopping counter: 5 out of 5
Early stopping
Training time: 4.212 minutes


100%|███████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 14.81it/s]


test accuracy 91.5
Elapsed time 4.396073114871979 mins.
